# CPSC 4970 AI + ML: Module 7 -- first sequence example

In [105]:
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer
import numpy as np

sequence = list("now is the time for all good men to come to the aid of their country")

def time_delayed(seq, delay):
    features = []
    targets = []
    for target_index in range(delay, len(seq)):
        tmp_features = []
        for feature_offset in range(delay-1, -1, -1):
            tmp_features.append(seq[target_index-feature_offset-1])
        features.append(tmp_features)
        targets.append(seq[target_index])
    return features, targets

X, y = time_delayed(sequence, 3)
print(np.array(X).shape)
print(y)

(65, 3)
[' ', 'i', 's', ' ', 't', 'h', 'e', ' ', 't', 'i', 'm', 'e', ' ', 'f', 'o', 'r', ' ', 'a', 'l', 'l', ' ', 'g', 'o', 'o', 'd', ' ', 'm', 'e', 'n', ' ', 't', 'o', ' ', 'c', 'o', 'm', 'e', ' ', 't', 'o', ' ', 't', 'h', 'e', ' ', 'a', 'i', 'd', ' ', 'o', 'f', ' ', 't', 'h', 'e', 'i', 'r', ' ', 'c', 'o', 'u', 'n', 't', 'r', 'y']


# Encoding

In [106]:

input_encoder = OneHotEncoder(sparse=False)
X_trans = input_encoder.fit_transform(X)
output_encoder = LabelBinarizer(sparse_output=False)
y_trans = output_encoder.fit_transform(y)

print(X_trans)
print(y_trans)

[[0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]]


# Network

In [107]:
from keras.losses import CategoricalCrossentropy
from keras.layers import Dense
from keras import Input, activations
from keras.models import Sequential
model = Sequential(
    [Input(shape=(X_trans.shape[1])),
     Dense(100, activation=activations.sigmoid),
     Dense(y_trans.shape[1], activation=activations.softmax)]
)
model.summary()
model.compile(optimizer='adam', loss=CategoricalCrossentropy(), metrics=['accuracy'])
model.fit(X_trans, y_trans, epochs=3000, verbose=True)

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_34 (Dense)             (None, 100)               5500      
_________________________________________________________________
dense_35 (Dense)             (None, 18)                1818      
Total params: 7,318
Trainable params: 7,318
Non-trainable params: 0
_________________________________________________________________
Epoch 1/3000
3/3 [==============================] - 0s 2ms/step - loss: 3.0209 - accuracy: 0.0462
Epoch 2/3000
3/3 [==============================] - 0s 1ms/step - loss: 2.9062 - accuracy: 0.0462
Epoch 3/3000
3/3 [==============================] - 0s 1ms/step - loss: 2.8228 - accuracy: 0.1077

In [108]:
def predict_from_seed(seed, model, output_encoder, input_encoder, prediction_count):
    result = seed
    new_seed = seed
    for i in range(prediction_count):
        p = output_encoder.inverse_transform(model.predict(input_encoder.transform([list(new_seed)])))
        result = result + p[0]
        new_seed = result[-len(seed):]
    return result

print(predict_from_seed("is ", model, 20))

is the aid of the aid o
